In [1230]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/e-commerce-shoppers-behaviour-understanding/test_data_v2.csv
/kaggle/input/e-commerce-shoppers-behaviour-understanding/train_data_v2.csv
/kaggle/input/e-commerce-shoppers-behaviour-understanding/sample.csv


In [1231]:
# GET THE DATA 
train_df = pd.read_csv('/kaggle/input/e-commerce-shoppers-behaviour-understanding/train_data_v2.csv')
test_df = pd.read_csv('/kaggle/input/e-commerce-shoppers-behaviour-understanding/test_data_v2.csv')

train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14731 entries, 0 to 14730
Data columns (total 22 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   HomePage                         14578 non-null  float64
 1   HomePage_Duration                14581 non-null  float64
 2   LandingPage                      14578 non-null  float64
 3   LandingPage_Duration             14596 non-null  float64
 4   ProductDescriptionPage           14608 non-null  float64
 5   ProductDescriptionPage_Duration  14564 non-null  float64
 6   GoogleMetric:Bounce Rates        14580 non-null  float64
 7   GoogleMetric:Exit Rates          14602 non-null  float64
 8   GoogleMetric:Page Values         14599 non-null  float64
 9   SeasonalPurchase                 14581 non-null  float64
 10  Month_SeasonalPurchase           14587 non-null  object 
 11  OS                               14597 non-null  float64
 12  SearchEngine      

In [1232]:
# train_df[train_df["Made_Purchase"]==True].count()
# The difference b/w True and False count is not much !!

In [1233]:
X= train_df.drop("Made_Purchase", axis= 1)
Y= train_df["Made_Purchase"]

In [1234]:
# Fill Null values

num_cols = X.select_dtypes(include = 'float64').columns
for column in num_cols:
    X[column].fillna( X[column].mean(),inplace = True)
    test_df[column].fillna( X[column].mean(),inplace = True)
    
cat_cols = X.select_dtypes(include = 'object').columns
for column in cat_cols:
    X[column].fillna(X[column].mode()[0],inplace = True)
    test_df[column].fillna(X[column].mode()[0],inplace = True)
# X.isna().sum()*100/(X.shape[0])

In [1235]:
# Encoding of cat variables

from sklearn.preprocessing import OneHotEncoder
def encoding(X):
    encoder= OneHotEncoder()
    encoded_arr= encoder.fit_transform(X[cat_cols]).toarray()
    encoded_cat= encoder.categories_
    encoded_cat= np.hstack(encoded_cat).ravel()
    col_df= pd.DataFrame(encoded_arr,columns=encoded_cat)
    return pd.concat([X[num_cols], col_df], axis=1,)

In [1236]:
X= encoding(X)
test_df= encoding(test_df)
X.columns

Index(['HomePage', 'HomePage_Duration', 'LandingPage', 'LandingPage_Duration',
       'ProductDescriptionPage', 'ProductDescriptionPage_Duration',
       'GoogleMetric:Bounce Rates', 'GoogleMetric:Exit Rates',
       'GoogleMetric:Page Values', 'SeasonalPurchase', 'OS', 'SearchEngine',
       'Zone', 'Type of Traffic', 'WeekendPurchase', 'Aug', 'Dec', 'Feb',
       'Jul', 'June', 'Mar', 'May', 'Nov', 'Oct', 'Sep', 'New_Visitor',
       'Other', 'Returning_Visitor', 'Female', 'Male', 'Not Specified', 'ALL',
       'Deny', 'Required', 'Diploma', 'Graduate', 'Not Specified', 'Others',
       'Married', 'Other', 'Single'],
      dtype='object')

In [1237]:
from sklearn.preprocessing import StandardScaler, QuantileTransformer, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer
import matplotlib.pylab as plt
from sklearn.utils import shuffle

**KNN Classifier**

In [1238]:
from sklearn.neighbors import KNeighborsClassifier
pipe= Pipeline(steps= [
    ("scalar", QuantileTransformer()),
    ("pca", PCA()),
    ("model", KNeighborsClassifier())
    ])
pipe.get_params()

{'memory': None,
 'steps': [('scalar', QuantileTransformer()),
  ('pca', PCA()),
  ('model', KNeighborsClassifier())],
 'verbose': False,
 'scalar': QuantileTransformer(),
 'pca': PCA(),
 'model': KNeighborsClassifier(),
 'scalar__copy': True,
 'scalar__ignore_implicit_zeros': False,
 'scalar__n_quantiles': 1000,
 'scalar__output_distribution': 'uniform',
 'scalar__random_state': None,
 'scalar__subsample': 100000,
 'pca__copy': True,
 'pca__iterated_power': 'auto',
 'pca__n_components': None,
 'pca__random_state': None,
 'pca__svd_solver': 'auto',
 'pca__tol': 0.0,
 'pca__whiten': False,
 'model__algorithm': 'auto',
 'model__leaf_size': 30,
 'model__metric': 'minkowski',
 'model__metric_params': None,
 'model__n_jobs': None,
 'model__n_neighbors': 5,
 'model__p': 2,
 'model__weights': 'uniform'}

In [1239]:
gscv_knn= GridSearchCV(estimator= pipe,
                   param_grid ={"model__n_neighbors":[5,6,7,8,9,10,11,12,13,14,]},
                   scoring={"f1_score":make_scorer(f1_score)},
                   refit="f1_score",
                   cv=4,
                   n_jobs=-1)

In [1240]:
X,Y = shuffle(X,Y, random_state=0)
gscv_knn.fit(X,Y)
pd.DataFrame(gscv_knn.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__n_neighbors,params,split0_test_f1_score,split1_test_f1_score,split2_test_f1_score,split3_test_f1_score,mean_test_f1_score,std_test_f1_score,rank_test_f1_score
0,0.169161,0.006765,1.594548,0.071519,5,{'model__n_neighbors': 5},0.284233,0.302799,0.314176,0.305603,0.301703,0.010922,1
1,0.157276,0.015108,1.392551,0.033689,6,{'model__n_neighbors': 6},0.205464,0.209474,0.206054,0.219757,0.210187,0.005733,10
2,0.161500,0.013813,1.446962,0.039115,7,{'model__n_neighbors': 7},0.291609,0.294595,0.307623,0.296629,0.297614,0.006048,2
3,0.158637,0.012083,1.483316,0.037550,8,{'model__n_neighbors': 8},0.213070,0.223642,0.221625,0.221267,0.219901,0.004047,9
4,0.157729,0.006556,1.516375,0.027926,9,{'model__n_neighbors': 9},0.283389,0.291280,0.300843,0.291902,0.291854,0.006181,4
5,0.165565,0.010269,1.455081,0.037774,10,{'model__n_neighbors': 10},0.221854,0.224249,0.234655,0.233010,0.228442,0.005487,8
6,0.162045,0.009660,1.457908,0.024600,11,{'model__n_neighbors': 11},0.285293,0.294988,0.301211,0.296225,0.294429,0.005766,3
7,0.157366,0.005036,1.464118,0.009166,12,{'model__n_neighbors': 12},0.226541,0.228415,0.228856,0.235746,0.229889,0.003491,7
8,0.154176,0.004700,1.492101,0.013350,13,{'model__n_neighbors': 13},0.284557,0.291300,0.290581,0.290114,0.289138,0.002678,5
9,0.156527,0.001332,1.482621,0.030047,14,{'model__n_neighbors': 14},0.231156,0.223930,0.235164,0.239250,0.232375,0.005654,6


**AdaBoostClassifier**

In [1241]:
from sklearn.ensemble import AdaBoostClassifier

In [1242]:
pipe= Pipeline(steps= [
    ("scalar2", QuantileTransformer()),
    ("pca", PCA()),
    ("model", AdaBoostClassifier(random_state=0))
    ])
pipe.get_params()

{'memory': None,
 'steps': [('scalar2', QuantileTransformer()),
  ('pca', PCA()),
  ('model', AdaBoostClassifier(random_state=0))],
 'verbose': False,
 'scalar2': QuantileTransformer(),
 'pca': PCA(),
 'model': AdaBoostClassifier(random_state=0),
 'scalar2__copy': True,
 'scalar2__ignore_implicit_zeros': False,
 'scalar2__n_quantiles': 1000,
 'scalar2__output_distribution': 'uniform',
 'scalar2__random_state': None,
 'scalar2__subsample': 100000,
 'pca__copy': True,
 'pca__iterated_power': 'auto',
 'pca__n_components': None,
 'pca__random_state': None,
 'pca__svd_solver': 'auto',
 'pca__tol': 0.0,
 'pca__whiten': False,
 'model__algorithm': 'SAMME.R',
 'model__base_estimator': None,
 'model__learning_rate': 1.0,
 'model__n_estimators': 50,
 'model__random_state': 0}

In [1243]:
gscv_adb= GridSearchCV(estimator= pipe,
                   param_grid ={"model__n_estimators":[400,500,600,800], "model__learning_rate":[0.1,0.01]},
                   scoring={"f1_score":make_scorer(f1_score)},
                   refit="f1_score",
                   cv=4,
                   n_jobs=-1)

In [1244]:
X,Y = shuffle(X,Y, random_state=0)
gscv_adb.fit(X,Y)
pd.DataFrame(gscv_adb.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__learning_rate,param_model__n_estimators,params,split0_test_f1_score,split1_test_f1_score,split2_test_f1_score,split3_test_f1_score,mean_test_f1_score,std_test_f1_score,rank_test_f1_score
0,36.108153,0.172866,0.505021,0.057239,0.1,400,"{'model__learning_rate': 0.1, 'model__n_estima...",0.406087,0.412150,0.389354,0.403931,0.402880,0.008370,2
1,44.512715,0.189928,0.560350,0.007757,0.1,500,"{'model__learning_rate': 0.1, 'model__n_estima...",0.406828,0.412698,0.389572,0.401180,0.402570,0.008538,4
2,53.570805,0.123978,0.663101,0.012574,0.1,600,"{'model__learning_rate': 0.1, 'model__n_estima...",0.407021,0.414339,0.387033,0.404505,0.403225,0.010022,1
3,71.110654,0.341792,0.876199,0.008872,0.1,800,"{'model__learning_rate': 0.1, 'model__n_estima...",0.405482,0.413054,0.389458,0.402737,0.402683,0.008519,3
4,36.049233,0.221514,0.458002,0.009498,0.01,400,"{'model__learning_rate': 0.01, 'model__n_estim...",0.298217,0.331269,0.254733,0.239583,0.280951,0.036155,8
5,45.037648,0.213201,0.568681,0.030406,0.01,500,"{'model__learning_rate': 0.01, 'model__n_estim...",0.304885,0.337423,0.267948,0.266742,0.294250,0.029264,7
6,53.443211,0.267116,0.669496,0.014934,0.01,600,"{'model__learning_rate': 0.01, 'model__n_estim...",0.313830,0.349270,0.301310,0.285395,0.312451,0.023525,6
7,71.116857,0.274395,0.736494,0.103367,0.01,800,"{'model__learning_rate': 0.01, 'model__n_estim...",0.334369,0.366765,0.314439,0.316366,0.332984,0.020995,5


**Voting Classifier**

In [1245]:
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier, GradientBoostingClassifier, RandomForestClassifier,ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

clf1 = LogisticRegression(random_state=0, warm_start=True)
clf2 = RidgeClassifier(random_state=0, solver="lsqr",  alpha=0.1)
clf3 = AdaBoostClassifier(n_estimators=500, learning_rate=0.01)
clf4= SVC()
clf5= LDA()

eclf = VotingClassifier(
     estimators=[ ('LogisticRegression', clf1),('RidgeClassifier', clf2), ('AdaBoostClassifier', clf3),
                 ('SVC', clf4), ('LDA', clf5)],
                 voting='hard')

In [1246]:
pipe= Pipeline(steps= [
    ("scalar", QuantileTransformer()),
    ("pca", PCA()),
    ("model", eclf)
    ])
# pipe.get_params()

In [1247]:
gscv_vc= GridSearchCV(estimator= pipe,
                   param_grid ={},
                   scoring={"f1_score":make_scorer(f1_score)},
                   refit="f1_score",
                   cv=4,)

In [1248]:
X,Y = shuffle(X,Y, random_state=0)
gscv_vc.fit(X,Y)
pd.DataFrame(gscv_vc.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,params,split0_test_f1_score,split1_test_f1_score,split2_test_f1_score,split3_test_f1_score,mean_test_f1_score,std_test_f1_score,rank_test_f1_score
0,42.026505,0.109695,3.392636,0.033167,{},0.420223,0.421351,0.412831,0.44363,0.424509,0.011514,1


**RidgeClassifier**

In [1249]:
from sklearn.linear_model import RidgeClassifier

In [1250]:
pipe= Pipeline(steps= [
    ("scalar", QuantileTransformer()),
    ("pca", PCA()),
    ("model", RidgeClassifier(random_state=0, solver="auto"))
    ])
pipe.get_params()

{'memory': None,
 'steps': [('scalar', QuantileTransformer()),
  ('pca', PCA()),
  ('model', RidgeClassifier(random_state=0))],
 'verbose': False,
 'scalar': QuantileTransformer(),
 'pca': PCA(),
 'model': RidgeClassifier(random_state=0),
 'scalar__copy': True,
 'scalar__ignore_implicit_zeros': False,
 'scalar__n_quantiles': 1000,
 'scalar__output_distribution': 'uniform',
 'scalar__random_state': None,
 'scalar__subsample': 100000,
 'pca__copy': True,
 'pca__iterated_power': 'auto',
 'pca__n_components': None,
 'pca__random_state': None,
 'pca__svd_solver': 'auto',
 'pca__tol': 0.0,
 'pca__whiten': False,
 'model__alpha': 1.0,
 'model__class_weight': None,
 'model__copy_X': True,
 'model__fit_intercept': True,
 'model__max_iter': None,
 'model__normalize': 'deprecated',
 'model__positive': False,
 'model__random_state': 0,
 'model__solver': 'auto',
 'model__tol': 0.001}

In [1251]:
gscv_rc= GridSearchCV(estimator=pipe,
                       param_grid ={"model__alpha":[0.1,0.01,0.001], },
                       scoring={"f1_score":make_scorer(f1_score)},
                       refit="f1_score",
                       cv=4,
                       n_jobs=-1)

In [1252]:
X,Y = shuffle(X,Y, random_state=0)
gscv_rc.fit(X,Y)
pd.DataFrame(gscv_rc.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__alpha,params,split0_test_f1_score,split1_test_f1_score,split2_test_f1_score,split3_test_f1_score,mean_test_f1_score,std_test_f1_score,rank_test_f1_score
0,0.185870,0.001610,0.026865,0.001409,0.1,{'model__alpha': 0.1},0.434701,0.40803,0.423023,0.435299,0.425263,0.011088,1
1,0.173021,0.000952,0.029775,0.000841,0.01,{'model__alpha': 0.01},0.434701,0.40803,0.423023,0.435299,0.425263,0.011088,1
2,0.178546,0.001747,0.025672,0.003363,0.001,{'model__alpha': 0.001},0.434701,0.40803,0.423023,0.435299,0.425263,0.011088,1


**Perceptron**

In [1253]:
from sklearn.linear_model import Perceptron
pipe= Pipeline(steps= [
    ("scalar", QuantileTransformer()),
    ("pca", PCA()),
    ("model", Perceptron(warm_start=True))
    ])
pipe.get_params()

{'memory': None,
 'steps': [('scalar', QuantileTransformer()),
  ('pca', PCA()),
  ('model', Perceptron(warm_start=True))],
 'verbose': False,
 'scalar': QuantileTransformer(),
 'pca': PCA(),
 'model': Perceptron(warm_start=True),
 'scalar__copy': True,
 'scalar__ignore_implicit_zeros': False,
 'scalar__n_quantiles': 1000,
 'scalar__output_distribution': 'uniform',
 'scalar__random_state': None,
 'scalar__subsample': 100000,
 'pca__copy': True,
 'pca__iterated_power': 'auto',
 'pca__n_components': None,
 'pca__random_state': None,
 'pca__svd_solver': 'auto',
 'pca__tol': 0.0,
 'pca__whiten': False,
 'model__alpha': 0.0001,
 'model__class_weight': None,
 'model__early_stopping': False,
 'model__eta0': 1.0,
 'model__fit_intercept': True,
 'model__l1_ratio': 0.15,
 'model__max_iter': 1000,
 'model__n_iter_no_change': 5,
 'model__n_jobs': None,
 'model__penalty': None,
 'model__random_state': 0,
 'model__shuffle': True,
 'model__tol': 0.001,
 'model__validation_fraction': 0.1,
 'model__ver

In [1254]:
gscv_pctron= GridSearchCV(estimator=pipe,
                       param_grid ={},
                       scoring={"f1_score":make_scorer(f1_score)},
                       refit="f1_score",
                       cv=20,
                       n_jobs=-1)

In [1255]:
X,Y = shuffle(X,Y, random_state=0)
gscv_pctron.fit(X,Y)
pd.DataFrame(gscv_pctron.cv_results_)
# Some f1_score are good but standard deviation is too much! (f1_score for some splits are very bad)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,params,split0_test_f1_score,split1_test_f1_score,split2_test_f1_score,split3_test_f1_score,split4_test_f1_score,...,split13_test_f1_score,split14_test_f1_score,split15_test_f1_score,split16_test_f1_score,split17_test_f1_score,split18_test_f1_score,split19_test_f1_score,mean_test_f1_score,std_test_f1_score,rank_test_f1_score
0,0.237253,0.019377,0.011516,0.001211,{},0.291765,0.304636,0.370618,0.295612,0.496075,...,0.433515,0.366472,0.440208,0.364583,0.396104,0.460751,0.47606,0.394794,0.061718,1


**Logistic Regression Classifier**

In [1256]:
from sklearn.linear_model import LogisticRegression
pipe= Pipeline(steps= [
    ("scalar", QuantileTransformer()),
    ("pca", PCA()),
    ("model", LogisticRegression())
    ])
pipe.get_params()

{'memory': None,
 'steps': [('scalar', QuantileTransformer()),
  ('pca', PCA()),
  ('model', LogisticRegression())],
 'verbose': False,
 'scalar': QuantileTransformer(),
 'pca': PCA(),
 'model': LogisticRegression(),
 'scalar__copy': True,
 'scalar__ignore_implicit_zeros': False,
 'scalar__n_quantiles': 1000,
 'scalar__output_distribution': 'uniform',
 'scalar__random_state': None,
 'scalar__subsample': 100000,
 'pca__copy': True,
 'pca__iterated_power': 'auto',
 'pca__n_components': None,
 'pca__random_state': None,
 'pca__svd_solver': 'auto',
 'pca__tol': 0.0,
 'pca__whiten': False,
 'model__C': 1.0,
 'model__class_weight': None,
 'model__dual': False,
 'model__fit_intercept': True,
 'model__intercept_scaling': 1,
 'model__l1_ratio': None,
 'model__max_iter': 100,
 'model__multi_class': 'auto',
 'model__n_jobs': None,
 'model__penalty': 'l2',
 'model__random_state': None,
 'model__solver': 'lbfgs',
 'model__tol': 0.0001,
 'model__verbose': 0,
 'model__warm_start': False}

In [1257]:
gscv_logr= GridSearchCV(estimator=pipe,
                       param_grid ={ "model__solver":["saga","sag","lbfgs", "newton-cg","liblinear"], "model__C": [1.0,0.1,0.01,5.0,]},
                       scoring={"f1_score":make_scorer(f1_score)},
                       refit="f1_score",
                       cv=4,
                       n_jobs=-1)

In [1258]:
X,Y = shuffle(X,Y, random_state=0)
gscv_logr.fit(X,Y)
pd.DataFrame(gscv_logr.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__C,param_model__solver,params,split0_test_f1_score,split1_test_f1_score,split2_test_f1_score,split3_test_f1_score,mean_test_f1_score,std_test_f1_score,rank_test_f1_score
0,0.448514,0.013773,0.028756,0.000805,1.0,saga,"{'model__C': 1.0, 'model__solver': 'saga'}",0.408814,0.419066,0.448513,0.421587,0.424495,0.014669,2
1,0.758013,0.041357,0.028621,0.001027,1.0,sag,"{'model__C': 1.0, 'model__solver': 'sag'}",0.408814,0.419066,0.448513,0.421587,0.424495,0.014669,2
2,0.220167,0.005135,0.032201,0.002016,1.0,lbfgs,"{'model__C': 1.0, 'model__solver': 'lbfgs'}",0.408814,0.419066,0.448513,0.421587,0.424495,0.014669,2
3,0.342039,0.015653,0.030171,0.001174,1.0,newton-cg,"{'model__C': 1.0, 'model__solver': 'newton-cg'}",0.408814,0.419066,0.448513,0.421587,0.424495,0.014669,2
4,0.372181,0.021311,0.028731,0.000920,1.0,liblinear,"{'model__C': 1.0, 'model__solver': 'liblinear'}",0.408814,0.419066,0.448513,0.421587,0.424495,0.014669,2
5,0.462279,0.016344,0.035008,0.010737,0.1,saga,"{'model__C': 0.1, 'model__solver': 'saga'}",0.408814,0.419066,0.447802,0.421587,0.424317,0.014378,12
6,0.461243,0.011042,0.027250,0.001092,0.1,sag,"{'model__C': 0.1, 'model__solver': 'sag'}",0.408814,0.419066,0.447802,0.421587,0.424317,0.014378,12
7,0.199685,0.006765,0.027779,0.000962,0.1,lbfgs,"{'model__C': 0.1, 'model__solver': 'lbfgs'}",0.408814,0.419066,0.447802,0.421587,0.424317,0.014378,12
8,0.300408,0.018358,0.028646,0.001022,0.1,newton-cg,"{'model__C': 0.1, 'model__solver': 'newton-cg'}",0.408814,0.419066,0.447802,0.421587,0.424317,0.014378,12
9,0.292536,0.009867,0.031070,0.001637,0.1,liblinear,"{'model__C': 0.1, 'model__solver': 'liblinear'}",0.409560,0.419066,0.447802,0.421587,0.424504,0.014179,1


**SGDClassifier**

In [1259]:
from sklearn.linear_model import SGDClassifier
pipe= Pipeline(steps= [
    ("scalar", QuantileTransformer()),
    ("pca", PCA()),
    ("model", SGDClassifier(max_iter= 5000, warm_start= True))
    ])
pipe.get_params()

{'memory': None,
 'steps': [('scalar', QuantileTransformer()),
  ('pca', PCA()),
  ('model', SGDClassifier(max_iter=5000, warm_start=True))],
 'verbose': False,
 'scalar': QuantileTransformer(),
 'pca': PCA(),
 'model': SGDClassifier(max_iter=5000, warm_start=True),
 'scalar__copy': True,
 'scalar__ignore_implicit_zeros': False,
 'scalar__n_quantiles': 1000,
 'scalar__output_distribution': 'uniform',
 'scalar__random_state': None,
 'scalar__subsample': 100000,
 'pca__copy': True,
 'pca__iterated_power': 'auto',
 'pca__n_components': None,
 'pca__random_state': None,
 'pca__svd_solver': 'auto',
 'pca__tol': 0.0,
 'pca__whiten': False,
 'model__alpha': 0.0001,
 'model__average': False,
 'model__class_weight': None,
 'model__early_stopping': False,
 'model__epsilon': 0.1,
 'model__eta0': 0.0,
 'model__fit_intercept': True,
 'model__l1_ratio': 0.15,
 'model__learning_rate': 'optimal',
 'model__loss': 'hinge',
 'model__max_iter': 5000,
 'model__n_iter_no_change': 5,
 'model__n_jobs': None,


In [1260]:
gscv_sgd= GridSearchCV(estimator=pipe,
                       param_grid ={"model__loss":["hinge", "log", "modified_huber", "squared_hinge", "perceptron"], 
                                    "model__alpha":[0.0001,0.001,0.01]},
                       scoring={"f1_score":make_scorer(f1_score)},
                       refit="f1_score",
                       cv=5,
                       n_jobs=-1)

In [1261]:
X,Y = shuffle(X,Y, random_state=0)
gscv_sgd.fit(X,Y)
pd.DataFrame(gscv_sgd.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__alpha,param_model__loss,params,split0_test_f1_score,split1_test_f1_score,split2_test_f1_score,split3_test_f1_score,split4_test_f1_score,mean_test_f1_score,std_test_f1_score,rank_test_f1_score
0,0.411703,0.022610,0.023683,0.001671,0.0001,hinge,"{'model__alpha': 0.0001, 'model__loss': 'hinge'}",0.389412,0.443817,0.427219,0.440421,0.424870,0.425148,0.019304,3
1,0.336682,0.024165,0.023451,0.002078,0.0001,log,"{'model__alpha': 0.0001, 'model__loss': 'log'}",0.379249,0.439108,0.411192,0.440092,0.415796,0.417087,0.022281,11
2,0.557835,0.068819,0.023288,0.001185,0.0001,modified_huber,"{'model__alpha': 0.0001, 'model__loss': 'modif...",0.366667,0.390006,0.243767,0.410876,0.440258,0.370315,0.067753,15
3,0.528059,0.033146,0.023371,0.001314,0.0001,squared_hinge,"{'model__alpha': 0.0001, 'model__loss': 'squar...",0.363529,0.496721,0.333333,0.487783,0.302703,0.396814,0.080314,14
4,0.315174,0.019118,0.023056,0.001358,0.0001,perceptron,"{'model__alpha': 0.0001, 'model__loss': 'perce...",0.444158,0.467466,0.393834,0.453379,0.264085,0.404584,0.074508,13
5,0.238312,0.009689,0.024135,0.000933,0.001,hinge,"{'model__alpha': 0.001, 'model__loss': 'hinge'}",0.389412,0.443817,0.427219,0.440421,0.424870,0.425148,0.019304,3
6,0.229972,0.010903,0.024050,0.001019,0.001,log,"{'model__alpha': 0.001, 'model__loss': 'log'}",0.389412,0.438073,0.426113,0.438050,0.424697,0.423269,0.017856,7
7,0.342183,0.006973,0.024436,0.001045,0.001,modified_huber,"{'model__alpha': 0.001, 'model__loss': 'modifi...",0.389183,0.445326,0.419183,0.436471,0.407628,0.419558,0.020075,10
8,0.320690,0.015070,0.023151,0.001474,0.001,squared_hinge,"{'model__alpha': 0.001, 'model__loss': 'square...",0.389671,0.443434,0.421615,0.432239,0.423543,0.422100,0.017955,8
9,0.218852,0.010831,0.022714,0.000796,0.001,perceptron,"{'model__alpha': 0.001, 'model__loss': 'percep...",0.434783,0.394299,0.478663,0.445356,0.284852,0.407591,0.067020,12


**Mlp Classifier**

In [1262]:
from sklearn.neural_network import MLPClassifier
pipe= Pipeline(steps= [
    ("scalar", QuantileTransformer()),
    ("pca", PCA()),
    ("model", MLPClassifier(random_state=0, max_iter=1000, activation="logistic"))
    ])
pipe.get_params()

{'memory': None,
 'steps': [('scalar', QuantileTransformer()),
  ('pca', PCA()),
  ('model',
   MLPClassifier(activation='logistic', max_iter=1000, random_state=0))],
 'verbose': False,
 'scalar': QuantileTransformer(),
 'pca': PCA(),
 'model': MLPClassifier(activation='logistic', max_iter=1000, random_state=0),
 'scalar__copy': True,
 'scalar__ignore_implicit_zeros': False,
 'scalar__n_quantiles': 1000,
 'scalar__output_distribution': 'uniform',
 'scalar__random_state': None,
 'scalar__subsample': 100000,
 'pca__copy': True,
 'pca__iterated_power': 'auto',
 'pca__n_components': None,
 'pca__random_state': None,
 'pca__svd_solver': 'auto',
 'pca__tol': 0.0,
 'pca__whiten': False,
 'model__activation': 'logistic',
 'model__alpha': 0.0001,
 'model__batch_size': 'auto',
 'model__beta_1': 0.9,
 'model__beta_2': 0.999,
 'model__early_stopping': False,
 'model__epsilon': 1e-08,
 'model__hidden_layer_sizes': (100,),
 'model__learning_rate': 'constant',
 'model__learning_rate_init': 0.001,
 'm

In [1263]:
gscv_mlp= GridSearchCV(estimator=pipe,
                       param_grid ={},
                       scoring={"f1_score":make_scorer(f1_score)},
                       refit="f1_score",
                       cv=4,
                       n_jobs=-1)

In [1264]:
X,Y = shuffle(X,Y, random_state=0)
gscv_mlp.fit(X,Y)
pd.DataFrame(gscv_mlp.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,params,split0_test_f1_score,split1_test_f1_score,split2_test_f1_score,split3_test_f1_score,mean_test_f1_score,std_test_f1_score,rank_test_f1_score
0,2.429293,0.256225,0.031111,0.006173,{},0.432608,0.437179,0.42037,0.411354,0.425378,0.010165,1


**Predicting the final test**

In [1265]:
best_model= gscv_vc
test_pred = best_model.predict(test_df)
Y_final= []
for i in test_pred:
    if i==0:
        Y_final.append(False)
    else:
        Y_final.append(True)

In [1266]:
submission=pd.DataFrame(columns=['id', 'Made_Purchase'])
submission['id']=[i for i in range(len(Y_final))]
submission["Made_Purchase"]=[i for i in Y_final]
submission.to_csv('submission.csv',index=False)
submission

,id,Made_Purchase
0,0,False
1,1,False
2,2,False
3,3,False
4,4,False
...,...,...
6594,6594,False
6595,6595,False
6596,6596,False
6597,6597,False
